In [636]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [637]:
filename = "ABox.ttl"

In [638]:
text1 = '''CQ_1.1
Return the title of the CHO in italian as given by the museum.
'''

query1 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>


SELECT ?item ?content
WHERE {
   ?item a lrmoo:F5_Item .
   ?manifestation lrmoo:R7i_is_exemplified_by ?item .
   ?expression lrmoo:R4i_is_embodied_in ?manifestation .
   ?work lrmoo:R3_is_realised_in ?expression ;
	 crm:P102_has_title ?title .
   ?title crm:P2_has_type aat:300417207 ;
       crm:P190_has_symbolic_content ?content .


FILTER( lang(?content) = "it")
}
'''

In [639]:
text2 = '''CQ_1.2
Return the type of the CHO.
'''

query2 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>


SELECT ?item ?type
WHERE {
   ?item a lrmoo:F5_Item .
   ?manifestation lrmoo:R7i_is_exemplified_by ?item ;
	crm:P2_has_type ?type .
}

'''

In [640]:
text3 = '''CQ_1.3
Return the description of the CHO.
'''

query3 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>

SELECT ?item ?description
WHERE {
   ?item a lrmoo:F5_Item ;
   	crm:P3_has_note ?description .
}

'''

In [641]:
text5 = '''CQ_2.1
Return the main technique used to create the CHO.
'''

query5 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>


SELECT ?item ?technique
WHERE {
   ?item a lrmoo:F5_Item .
   ?manifestation lrmoo:R7i_is_exemplified_by ?item .
   ?expression lrmoo:R4i_is_embodied_in ?manifestation .
   ?creation a lrmoo:F28_Expression_Creation ;
     lrmoo:R17_created ?expression ;
     crm:P32_used_general_technique ?technique .
}

'''

In [642]:
text6 = '''CQ_2.2
Return the date of creation of the CHO.
'''

query6 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>


SELECT ?item (CONCAT(CONCAT(STR(MONTH(?begin)), "/", STR(DAY(?begin)), "/", STR(YEAR(?begin))), "-", CONCAT(STR(MONTH(?end)), "/", STR(DAY(?end)), "/", STR(YEAR(?end)))) AS ?date)
WHERE
{
   ?item a lrmoo:F5_Item .
   ?manifestation lrmoo:R7i_is_exemplified_by ?item .
   ?expression lrmoo:R4i_is_embodied_in ?manifestation .
   ?creation a lrmoo:F28_Expression_Creation ;
      lrmoo:R17_created ?expression ;
      crm:P4_has_time-span ?timespan .
   ?timespan crm:P82a_begin_of_the_begin ?begin ;
      crm:P82b_end_of_the_end ?end .
} 

'''

In [643]:
text7 = '''CQ_2.3
Return the organization responsible with the curation of the CHO.
'''

query7 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>


SELECT ?item ?conservation_org
WHERE {
   ?item a lrmoo:F5_Item .
   ?activity crm:P16_used_specific_object ?item ;
	 crm:P2_has_type aat:300054277 ;
 crm:P14_carried_out_by ?conservation_org .
}

'''

In [644]:
text8 = '''CQ_3.1
Return the place where the CHO is conserved.
'''

query8 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>


SELECT ?item ?conservation_place
WHERE {
   ?item a lrmoo:F5_Item .
   ?activity crm:P16_used_specific_object ?item ;
	 crm:P2_has_type aat:300054277 ;
 crm:P14_carried_out_by ?conservation_org .
   ?conservation_org crm:P74_has_current_or_former_residence ?conservation_place .
}

'''

In [645]:
text10 = '''CQ_3.1
Return the existing digital representation of the CHO.
'''

query10 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>


SELECT ?item ?digital_repr
WHERE {
   ?item a lrmoo:F5_Item .
   ?manifestation lrmoo:R7i_is_exemplified_by ?item ;
       crm:P130i_features_are_also_found_on ?digital_repr .
}

'''

In [646]:
text12 = '''CQ_5.1
Return the existing 3D model of the CHO.
'''

query12 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>


SELECT ?item ?existing_model
WHERE {
   ?item a lrmoo:F5_Item ;
       crm:P130i_features_are_also_found_on ?3d_model .
   ?3d_model a crmdig:D9_Data_Object ;
       crm:P130_shows_features_of ?existing_model .
}
'''

In [647]:
text15 = '''CQ_5.4
Return the subjects of the CHO.
'''

query15 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>


SELECT ?item (GROUP_CONCAT(?subject; SEPARATOR="; ") AS ?subjects)
WHERE {
   ?item a lrmoo:F5_Item .
   ?manifestation lrmoo:R7i_is_exemplified_by ?item .
   ?expression lrmoo:R4i_is_embodied_in ?manifestation ;
       crm:P129_is_about ?subject .
}

'''

In [648]:
text18 = '''CQ_5.7
Return the people and organizations involved in the creation of the CHO.
'''

query18 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>


SELECT ?item (GROUP_CONCAT(CONCAT(STR(?agent), " (", STR(?activity_type), ")"); SEPARATOR="; ") AS ?agents)
WHERE {
   ?item a lrmoo:F5_Item .
   ?manifestation lrmoo:R7i_is_exemplified_by ?item .
   ?expression lrmoo:R4i_is_embodied_in ?manifestation .
   ?creation a lrmoo:F28_Expression_Creation ;
      lrmoo:R17_created ?expression ;
      crm:P9_consists_of ?activity .
   ?activity crm:P2_has_type ?activity_type ;
	    crm:P14_carried_out_by ?agent .
}

'''

In [649]:
text19 = '''CQ_1.1
Return the link to the 3D model published on Aton.
'''

query19 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>

SELECT ?item ?link_aton
WHERE {
	?item a lrmoo:F5_Item .
	OPTIONAL {
		?acquisition crmdig:L1_digitized ?item ;
			crmdig:L11_had_output ?3d_model_00 .
	}
	OPTIONAL {
		?item crm:P130i_features_are_also_found_on ?3d_model_00 .
	}
	?3d_model_00 (^crmdig:L10_had_input/crmdig:L11_had_output)+ ?3d_model .
	?3d_model crm:P1_is_identified_by ?id_aton .
	?id_aton crm:P190_has_symbolic_content ?link_aton .
}

'''

In [650]:
text20 = '''CQ_1.2
Return the license associated with the published 3D model of the CHO.'''

query20 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>

SELECT ?item ?license_link
WHERE {
    ?item a lrmoo:F5_Item .
	OPTIONAL {
		?acquisition crmdig:L1_digitized ?item ;
			crmdig:L11_had_output ?3d_model_00 .
	}
	OPTIONAL {
		?item crm:P130i_features_are_also_found_on ?3d_model_00 .
	}
	?3d_model_00 (^crmdig:L10_had_input/crmdig:L11_had_output)+ ?3d_model .
    ?license crm:P2_has_type aat:300435434 ;
        crm:P67_refers_to ?3d_model ;
        crm:P70i_is_documented_in ?license_link .
}
GROUP BY ?item

'''

In [651]:
queries = [(text1, query1),
           (text2, query2),
           (text3, query3),
           (text5, query5),
           (text6, query6),
           (text7, query7),
           (text8, query8),
           (text10, query10),
           (text12, query12),
           (text15, query15),
           (text18, query18),
           (text19, query19),
           (text20, query20)]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))
    print("\n")

CQ_1.1
Return the title of the CHO in italian as given by the museum.

+---------------------------------------------------+--------------------------+
| item                                              | content                  |
|---------------------------------------------------+--------------------------|
| https://w3id.org/changes/4/aldrovandi/32/itm/00/1 | Essere umano ermafrodita |
+---------------------------------------------------+--------------------------+


CQ_1.2
Return the type of the CHO.

+---------------------------------------------------+-------------------------------------------+
| item                                              | type                                      |
|---------------------------------------------------+-------------------------------------------|
| https://w3id.org/changes/4/aldrovandi/32/itm/00/1 | http://vocab.getty.edu/page/aat/300028569 |
+---------------------------------------------------+-----------------------------------------